In [1]:
import pandas as pd

In [2]:
df_baseSNII = pd.read_csv('datasetF.csv')
df_baseSNII

,AÑO,CVU,NOBILIS,NOMBRE DEL INVESTIGADOR,INVESTIGADOR,NIVEL,FECHA DE INICIO DE VIGENCIA,FECHA DE FIN DE VIGENCIA,ÁREA DEL CONOCIMIENTO,DISCIPLINA,SUBDISCIPLINA,ESPECIALIDAD,INSTITUCIÓN DE ADSCRIPCIÓN,DEPENDENCIA,ENTIDAD FEDERATIVA,PAÍS
0,2003,19177.0,DR.,"ABDULLAEV JAFAROVA,FIKRAT","ABDULLAEV JAFAROVA,FIKRAT",1,2001-07-01,2004-06-30,II.- BIOLOGÍA Y QUÍMICA,BIOLOGIA,BIOLOGIA MOLECULAR,AGENTES ANTITUMORALES DE ORIGEN NATURAL Y SINT...,SECRETARIA DE SALUD,INSTITUTO NACIONAL DE PEDIATRIA,CIUDAD DE MÉXICO,MÉXICO
1,2003,13070.0,DR.,"ABITIA CARDENAS,LEONARDO ANDRES","ABITIA CARDENAS,LEONARDO ANDRES",1,2002-07-01,2005-06-30,VI.- BIOTECNOLOGÍA Y CIENCIAS AGROPECUARIAS,PESCA,PROSPECCION DE RECURSOS,"PESCA, ICTIOLOGIA MARINA",INSTITUTO POLITÉCNICO NACIONAL,CENTRO INTERDISCIPLINARIO DE CIENCIAS MARINAS,BAJA CALIFORNIA SUR,MÉXICO
2,2003,5885.0,DR.,"ABOITES AGUILAR,JAIME","ABOITES AGUILAR,JAIME",2,2001-07-01,2004-06-30,V.- CIENCIAS SOCIALES,ECONOMIA,ECONOMIA INDUSTRIAL,DESARROLLO TECNOLOGICO,UNIVERSIDAD AUTONOMA METROPOLITANA,DIVISION DE CIENCIAS SOCIALES Y HUMANIDADES. X...,CIUDAD DE MÉXICO,MÉXICO
3,2003,8479.0,DR.,"ABOITES AGUILAR,LUIS","ABOITES AGUILAR,LUIS",2,2001-07-01,2004-06-30,IV.- HUMANIDADES Y CIENCIAS DE LA CONDUCTA,HISTORIA,HISTORIA SOCIAL,HISTORIA DE LOS APROVECHAMIENTOS HIDRáULICOS,"EL COLEGIO DE MEXICO, A.C.",CENTRO DE ESTUDIOS HISTORICOS,CIUDAD DE MÉXICO,MÉXICO
4,2003,10132.0,DR.,"ABOITES MANRIQUE,GILBERTO","ABOITES MANRIQUE,GILBERTO",1,2001-07-01,2004-06-30,V.- CIENCIAS SOCIALES,SOCIOLOGIA,SOCIOLOGIA RURAL,NaN,UNIVERSIDAD AUTONOMA DE COAHUILA,No Disponible,COAHUILA DE ZARAGOZA,MÉXICO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461072,2023,1247488.0,MTRO.,"ESQUEDA MENDOZA,ANTONIO","ESQUEDA MENDOZA,ANTONIO",C,2023-01-01 00:00:00,2026-12-31 00:00:00,III.- MEDICINA Y CIENCIAS DE LA SALUD,CIRUGÍA,UROLOGÍA,CANCERUROLOGICO,SECRETARIA DE SALUD,SECRETARIA DE SALUD,YUCATÁN,MÉXICO
461073,2023,1247675.0,DRA.,"BANDIERA,ANTONELLA AGOSTINA","BANDIERA,ANTONELLA AGOSTINA",C,2023-01-01 00:00:00,2026-12-31 00:00:00,VI.- CIENCIAS SOCIALES,CIENCIAS POLÍTICAS,OTRAS,MIGRACION,INSTITUTO TECNOLOGICO AUTONOMO DE MEXICO A.C. ...,"DIVISION ACADEMICA DE ECONOMIA, DERECHO Y CIEN...",CIUDAD DE MÉXICO,MÉXICO
461074,2023,1248008.0,DR.,"TORRES ERAZO,DARWIN STALIN","TORRES ERAZO,DARWIN STALIN",C,2023-01-01 00:00:00,2026-12-31 00:00:00,III.- MEDICINA Y CIENCIAS DE LA SALUD,INFECTOLOGÍA,INFECTOLOGÍA,ESPECIALIDADENMEDICINA,SECRETARIA DE SALUD,SECRETARIA DE SALUD,YUCATÁN,MÉXICO
461075,2023,1248024.0,LIC,"ZAMARRON LOPEZ,EDER IVAN","ZAMARRON LOPEZ,EDER IVAN",C,2023-01-01 00:00:00,2026-12-31 00:00:00,III.- MEDICINA Y CIENCIAS DE LA SALUD,MEDICINA DEL ENFERMO EN ESTADO CRÍTICO,MEDICINA DEL ENFERMO EN ESTADO CRÍTICO,EMERGENCIAS,INSTITUTO MEXICANO DEL SEGURO SOCIAL,OOAD TAMAULIPAS,TAMAULIPAS,MÉXICO


In [3]:
df_basePATENTE = pd.read_csv('InventoresPatentes.csv')

# Convertimos Inventor en mayusculas
df_basePATENTE['Inventor'] = df_basePATENTE['Inventor'].str.upper()

df_basePATENTE

,Inventor,NOMBRE_INVENTOR,Patents,Cites,Asignee Inst,Número de coinventores,Co inventoras mujeres,Coinventores extranjeros,Location,years active,...,# cédula profesional MAESTRÍA,Año de expedición de cedula profesional MAESTRÍA,Grado académico DOCTORADO,Institución que otorgó el grado DOCTORADO,# cédula profesional DOCTORADO,Año de expedición de cedula profesional DOCTORADO,Posgrado SI= 1 NO= 0,Empleo actual,Puesto,Nacionalidad
0,RICK A HAMILTON II,"Hamilton, II,Rick A",1066,NaN,NaN,NaN,NaN,NaN,"Charlottesville, US",1999-2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,STUART C SALTER,"Salter,Stuart C",737,NaN,NaN,NaN,NaN,NaN,"White Lake, US",1990-2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GREGORY J BOSS,"Boss,Gregory J",608,NaN,NaN,NaN,NaN,NaN,"Saginaw, US",2008-2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BRIAN M OCONNELL,"O'Connell,Brian M",496,NaN,NaN,NaN,NaN,NaN,"Cary, US",2009-2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PAUL KENNETH DELLOCK,"Dellock,Paul Kenneth",488,NaN,NaN,NaN,NaN,NaN,"Northville, US",2002-2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11281,ZAIR DOMINGUEZ TRINIDAD,"Trinidad,Zair Dominguez",1,NaN,NaN,NaN,NaN,NaN,"Xalapa, MX",2021-2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11282,ZEFERINO RENDON,"Rendon,Zeferino",1,NaN,NaN,NaN,NaN,NaN,"Chihuahua, MX",2001-2001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11283,ZHENGLONG WU,"Wu,Zhenglong",1,NaN,NaN,NaN,NaN,NaN,"Shanghai, CN",2020-2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11284,ZORAM FLORENCIO,"Florencio,Zoram",1,NaN,NaN,NaN,NaN,NaN,"Tultepec, MX",2017-2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_lista = pd.read_csv('Nombres_PxS.csv')
df_lista

,NOMBRE_INVENTOR,NOMBRE DEL INVESTIGADOR,Ratio,inventor_id,CVU
0,"LOPEZ,NYDIA","LOPEZ LIRA,NIDIA",73.0,NaN,404438.0
1,"STEPHENS,CHRISTOPER R","STEPHENS STEVENS,CHRISTOPHER RHODES",73.0,NaN,16099.0
2,"STEPHENS,CHRISTOPER","STEPHENS STEVENS,CHRISTOPHER RHODES",73.0,NaN,16099.0
3,"GARCIA,MARCO","GARCIA JUAREZ,MARCOS",77.0,NaN,273360.0
4,"MORENO,MARCOS","MORENO IBARRA,MARCO",77.0,fl:ma_ln:pastor-1,17341.0
...,...,...,...,...,...
934,"AGUILAR ARMENDARIZ,RAMON ALBERTO","AGUILAR ARMENDARIZ,RAMON ALBERTO",100.0,NaN,30200.0
935,"SACRISTAN ROCK,EMILIO","SACRISTAN ROCK,EMILIO",100.0,NaN,16445.0
936,"ALONSO VILATELA,MARIA ELISA","ALONSO VILATELA,MARIA ELISA",100.0,NaN,6452.0
937,"VILLARREAL TREVINO,JUAN ANTONIO","VILLARREAL TREVINO,JUAN ANTONIO",100.0,fl:ju_ln:villarrealtrevino-1,32126.0


In [5]:
df_idPatentes = pd.read_csv('inventores_con_id.csv')

# Convertimos Inventor en mayusculas
df_idPatentes['Inventor'] = df_idPatentes['Inventor'].str.upper()

df_idPatentes

,Inventor,inventor_id
0,AARÓN FERNANDO GONZÁLEZ CÓRDOVA,fl:aa_ln:gonzalezcordova-1
1,ABRAHAM DUENAS DE LA CRUZ,fl:ab_ln:duenasdelacruz-1
2,ABRAHAM GÓMEZ LÓPEZ,fl:ab_ln:lopez-2
3,ADMA ELENA FARAON CHAUL HUERTA,fl:ad_ln:faraonchaulhuerta-1
4,ADRIÁN ENRIQUE MACOUZET DÍAZ LEAL,fl:ad_ln:macouzetdiazleal-1
...,...,...
1297,JOSE R. MOSQUEDA MEJIA,fl:jo_ln:mosquedamejia-1
1298,HECTOR CORDOURIER MARURI,fl:he_ln:cordouriermaruri-1
1299,DANIEL FELIPE ORTEGA,fl:da_ln:ortega-5
1300,EDUARDO CORONADO,fl:ed_ln:coronado-1


---

In [6]:
# Vamos a aliminar la columna 'a' del df_idPatentes
# df_idPatentes.drop('a', axis=1, inplace=True)

# Normalizamos los acentos,puntos, guion, y las letras que tengan diéresis como ö,en el 'df_idPatentes' en la columna 'Inventor'
df_idPatentes['Inventor'] = df_idPatentes['Inventor'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

df_idPatentes

,Inventor,inventor_id
0,AARON FERNANDO GONZALEZ CORDOVA,fl:aa_ln:gonzalezcordova-1
1,ABRAHAM DUENAS DE LA CRUZ,fl:ab_ln:duenasdelacruz-1
2,ABRAHAM GOMEZ LOPEZ,fl:ab_ln:lopez-2
3,ADMA ELENA FARAON CHAUL HUERTA,fl:ad_ln:faraonchaulhuerta-1
4,ADRIAN ENRIQUE MACOUZET DIAZ LEAL,fl:ad_ln:macouzetdiazleal-1
...,...,...
1297,JOSE R. MOSQUEDA MEJIA,fl:jo_ln:mosquedamejia-1
1298,HECTOR CORDOURIER MARURI,fl:he_ln:cordouriermaruri-1
1299,DANIEL FELIPE ORTEGA,fl:da_ln:ortega-5
1300,EDUARDO CORONADO,fl:ed_ln:coronado-1


In [7]:
# Vamos a hacer que se busque el nombre del inventor en la lista de nombres de la columna 'Inventor' del 'df_idPatentes' en la columna 'Inventor' del 'df_basePATENTE'
# si se encuentra el nombre en la lista, se va a agregar una nueva columna en el 'df_basePATENTE' llamada 'inventor_id' con el id del inventor que se encontró en la lista
df_basePATENTE['inventor_id'] = df_basePATENTE['Inventor'].apply(lambda x: df_idPatentes[df_idPatentes['Inventor'].str.contains(x, case=False, na=False)]['inventor_id'].values[0] if len(df_idPatentes[df_idPatentes['Inventor'].str.contains(x, case=False, na=False)]) > 0 else None)

df_basePATENTE

,Inventor,NOMBRE_INVENTOR,Patents,Cites,Asignee Inst,Número de coinventores,Co inventoras mujeres,Coinventores extranjeros,Location,years active,...,Año de expedición de cedula profesional MAESTRÍA,Grado académico DOCTORADO,Institución que otorgó el grado DOCTORADO,# cédula profesional DOCTORADO,Año de expedición de cedula profesional DOCTORADO,Posgrado SI= 1 NO= 0,Empleo actual,Puesto,Nacionalidad,inventor_id
0,RICK A HAMILTON II,"Hamilton, II,Rick A",1066,NaN,NaN,NaN,NaN,NaN,"Charlottesville, US",1999-2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,STUART C SALTER,"Salter,Stuart C",737,NaN,NaN,NaN,NaN,NaN,"White Lake, US",1990-2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,GREGORY J BOSS,"Boss,Gregory J",608,NaN,NaN,NaN,NaN,NaN,"Saginaw, US",2008-2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,BRIAN M OCONNELL,"O'Connell,Brian M",496,NaN,NaN,NaN,NaN,NaN,"Cary, US",2009-2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,PAUL KENNETH DELLOCK,"Dellock,Paul Kenneth",488,NaN,NaN,NaN,NaN,NaN,"Northville, US",2002-2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11281,ZAIR DOMINGUEZ TRINIDAD,"Trinidad,Zair Dominguez",1,NaN,NaN,NaN,NaN,NaN,"Xalapa, MX",2021-2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
11282,ZEFERINO RENDON,"Rendon,Zeferino",1,NaN,NaN,NaN,NaN,NaN,"Chihuahua, MX",2001-2001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
11283,ZHENGLONG WU,"Wu,Zhenglong",1,NaN,NaN,NaN,NaN,NaN,"Shanghai, CN",2020-2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
11284,ZORAM FLORENCIO,"Florencio,Zoram",1,NaN,NaN,NaN,NaN,NaN,"Tultepec, MX",2017-2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [8]:
# Vamos a buscar en el 'df_lista' los nombres de los inventores que se encuentran en la columna 'NOMBRE_INVENTOR' del 
# 'df_basePATENTE' y si se encuentra el nombre en la lista, se va a agregar una nueva columna en el 'df_lista' 
# llamada 'inventor_id' con el id del inventor que se encontró en 'df_basePATENTE'
df_lista.loc[:, 'inventor_id'] = df_lista['NOMBRE_INVENTOR'].apply(lambda x: df_basePATENTE[df_basePATENTE['NOMBRE_INVENTOR'].str.contains(x, case=False, na=False)]['inventor_id'].values[0] if len(df_basePATENTE[df_basePATENTE['NOMBRE_INVENTOR'].str.contains(x, case=False, na=False)]) > 0 else None)

df_lista

,NOMBRE_INVENTOR,NOMBRE DEL INVESTIGADOR,Ratio,inventor_id,CVU
0,"LOPEZ,NYDIA","LOPEZ LIRA,NIDIA",73.0,None,404438.0
1,"STEPHENS,CHRISTOPER R","STEPHENS STEVENS,CHRISTOPHER RHODES",73.0,None,16099.0
2,"STEPHENS,CHRISTOPER","STEPHENS STEVENS,CHRISTOPHER RHODES",73.0,None,16099.0
3,"GARCIA,MARCO","GARCIA JUAREZ,MARCOS",77.0,None,273360.0
4,"MORENO,MARCOS","MORENO IBARRA,MARCO",77.0,fl:ma_ln:pastor-1,17341.0
...,...,...,...,...,...
934,"AGUILAR ARMENDARIZ,RAMON ALBERTO","AGUILAR ARMENDARIZ,RAMON ALBERTO",100.0,None,30200.0
935,"SACRISTAN ROCK,EMILIO","SACRISTAN ROCK,EMILIO",100.0,None,16445.0
936,"ALONSO VILATELA,MARIA ELISA","ALONSO VILATELA,MARIA ELISA",100.0,None,6452.0
937,"VILLARREAL TREVINO,JUAN ANTONIO","VILLARREAL TREVINO,JUAN ANTONIO",100.0,NaN,32126.0


In [9]:
"""
Celda generada por Data Wrangler.
"""
def clean_data(df_lista):
    # Split and rearrange 'NOMBRE DEL INVESTIGADOR' into 'nombre' column
    df_lista['NOMBRE SNII'] = df_lista['NOMBRE DEL INVESTIGADOR'].apply(
        lambda x: ' '.join(reversed(x.split(',')))
    )
    df_lista['NOMBRE PATENTE'] = df_lista['NOMBRE_INVENTOR'].apply(
        lambda x: ' '.join(reversed(x.split(',')))
    )
    return df_lista

df_lista = clean_data(df_lista.copy())
df_lista.head()

,NOMBRE_INVENTOR,NOMBRE DEL INVESTIGADOR,Ratio,inventor_id,CVU,NOMBRE SNII,NOMBRE PATENTE
0,"LOPEZ,NYDIA","LOPEZ LIRA,NIDIA",73.0,None,404438.0,NIDIA LOPEZ LIRA,NYDIA LOPEZ
1,"STEPHENS,CHRISTOPER R","STEPHENS STEVENS,CHRISTOPHER RHODES",73.0,None,16099.0,CHRISTOPHER RHODES STEPHENS STEVENS,CHRISTOPER R STEPHENS
2,"STEPHENS,CHRISTOPER","STEPHENS STEVENS,CHRISTOPHER RHODES",73.0,None,16099.0,CHRISTOPHER RHODES STEPHENS STEVENS,CHRISTOPER STEPHENS
3,"GARCIA,MARCO","GARCIA JUAREZ,MARCOS",77.0,None,273360.0,MARCOS GARCIA JUAREZ,MARCO GARCIA
4,"MORENO,MARCOS","MORENO IBARRA,MARCO",77.0,fl:ma_ln:pastor-1,17341.0,MARCO MORENO IBARRA,MARCOS MORENO


In [10]:
df_lista['inventor_id'] = df_lista['inventor_id'].astype(object)

df_lista.loc[:, 'inventor_id'] = df_lista['NOMBRE PATENTE'].apply(lambda x: df_basePATENTE[df_basePATENTE['Inventor'].str.contains(x, case=False, na=False)]['inventor_id'].values[0] if len(df_basePATENTE[df_basePATENTE['Inventor'].str.contains(x, case=False, na=False)]) > 0 else None)

df_lista

,NOMBRE_INVENTOR,NOMBRE DEL INVESTIGADOR,Ratio,inventor_id,CVU,NOMBRE SNII,NOMBRE PATENTE
0,"LOPEZ,NYDIA","LOPEZ LIRA,NIDIA",73.0,None,404438.0,NIDIA LOPEZ LIRA,NYDIA LOPEZ
1,"STEPHENS,CHRISTOPER R","STEPHENS STEVENS,CHRISTOPHER RHODES",73.0,None,16099.0,CHRISTOPHER RHODES STEPHENS STEVENS,CHRISTOPER R STEPHENS
2,"STEPHENS,CHRISTOPER","STEPHENS STEVENS,CHRISTOPHER RHODES",73.0,None,16099.0,CHRISTOPHER RHODES STEPHENS STEVENS,CHRISTOPER STEPHENS
3,"GARCIA,MARCO","GARCIA JUAREZ,MARCOS",77.0,None,273360.0,MARCOS GARCIA JUAREZ,MARCO GARCIA
4,"MORENO,MARCOS","MORENO IBARRA,MARCO",77.0,fl:ma_ln:pastor-1,17341.0,MARCO MORENO IBARRA,MARCOS MORENO
...,...,...,...,...,...,...,...
934,"AGUILAR ARMENDARIZ,RAMON ALBERTO","AGUILAR ARMENDARIZ,RAMON ALBERTO",100.0,fl:ra_ln:armendariz-1,30200.0,RAMON ALBERTO AGUILAR ARMENDARIZ,RAMON ALBERTO AGUILAR ARMENDARIZ
935,"SACRISTAN ROCK,EMILIO","SACRISTAN ROCK,EMILIO",100.0,fl:em_ln:rock-1,16445.0,EMILIO SACRISTAN ROCK,EMILIO SACRISTAN ROCK
936,"ALONSO VILATELA,MARIA ELISA","ALONSO VILATELA,MARIA ELISA",100.0,fl:ma_ln:vilatela-1,6452.0,MARIA ELISA ALONSO VILATELA,MARIA ELISA ALONSO VILATELA
937,"VILLARREAL TREVINO,JUAN ANTONIO","VILLARREAL TREVINO,JUAN ANTONIO",100.0,NaN,32126.0,JUAN ANTONIO VILLARREAL TREVINO,JUAN ANTONIO VILLARREAL TREVINO


In [ ]:
# Vamos a buscar en el 'df_lista' los nombres de los inventores que se encuentran en la columna 'INVESTIGADOR' 
# del 'df_baseSNII' y si se encuentra el nombre en la lista, se va a agregar una nueva columna en el 'df_lista' 
# llamada 'CVU' con el id del inventor que se encontró en 'df_baseSNII'
df_lista['CVU'] = df_lista['NOMBRE DEL INVESTIGADOR'].apply(lambda x: df_baseSNII[df_baseSNII['INVESTIGADOR'].str.contains(x, case=False, na=False)]['CVU'].values[0] if len(df_baseSNII[df_baseSNII['INVESTIGADOR'].str.contains(x, case=False, na=False)]) > 0 else None)

df_lista

In [11]:
# guardamos el resultado en un archivo csv
df_lista.to_csv('Nombres_PxS.csv', index=False)